<a href="https://colab.research.google.com/github/BritCrit/Default-Flashtalking-Flag-Report/blob/main/Default_Flashtalking_Flag_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Upload Flashtalking Monthly Report for Default { vertical-output: true }
default_flag_threshold =   0.1 #@param {type: "number"}
!rm -rf sample_data/
!rm -rf Reports/
region = "SER" #@param ['SCR', 'SER', 'NER', 'WEST', 'NCR']
import os
import io
import pandas
import zipfile
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

with zipfile.ZipFile(io.BytesIO(uploaded[fn]), 'r') as zip_ref:
  zip_ref.extractall()



def create_dir():
  new_dir = os.path.join(os.sep, "/content/Reports/")
  if not os.path.exists(new_dir):
    os.mkdir(new_dir)

def zip_reports():
  !zip -r /content/reports.zip /content/
  files.download("/content/reports.zip")

def default_flag_report(df, region, default_flag_threshold):
  regions = {'_SER_' : 'SER', '_WER_': 'WER', '_NCR_': 'NCR', '_NER_': 'NER', '_SCR_' : 'SCR'}
  df = df[['campaign_name', 'creative', 'total_ads_requested']]
  # region mapping
  df.loc[:, 'region'] = df['campaign_name'].apply(lambda x: next((v for k, v in regions.items() if k in x), None))

  # Create a new column 'default' based on the 'creative' column
  df.loc[:, 'default'] = df['creative'].apply(lambda x: 1 if 'DLITE' in x else 0)

  df = df.groupby(['region','campaign_name'])[['total_ads_requested','default']].sum().reset_index()

  df['default_rate'] = df['default'] / df['total_ads_requested']

  df['default_flag'] = df['default_rate'] > default_flag_threshold

  default_flags_report = df[(df['region'] == region )& (df['default_flag'] == True)]

  df.to_csv('Reports/all_regions_default_flag_report.csv', index=False)

  default_flags_report.to_csv(f'Reports/{region}_default_flags_report.csv', index=False)

create_dir()

df = pd.read_csv(zip_ref.namelist()[0])

default_flag_report(df, region, default_flag_threshold)

zip_reports()

